In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time, sys
from IPython.display import clear_output

# Utilities

In [ ]:
def updateProgress(progress, message=''):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    if(message): print(message)
    print(text)

In [ ]:
def imgshow1(image, title):
    fig = plt.figure()
    plt.title(title)
    plt.axis('off')
    (h, w, *c) = image.shape
    image_abs = np.abs(image)
    if(len(c) > 0):
        image_abs = cv2.cvtColor(image_abs, cv2.COLOR_BGR2RGB)
        plt.imshow(image_abs)
    else:
        plt.imshow(image_abs, cmap = 'gray')

In [ ]:
def imgshow(images_arr, titles_arr, S=4):
    images = np.array(images_arr)
    titles = np.array(titles_arr)
    (nY, nX, *rest) = images.shape
    figsize = S * nX, S * nY
    fig = plt.figure(figsize=figsize)
    plt.axis('off')
    for yy in range(nY):
        for xx in range(nX):
            fig.add_subplot(nY, nX, yy * nX + xx + 1)
            plt.title(titles[yy, xx])
            plt.axis('off')
            (h, w, *c) = images[yy, xx].shape
            img_abs = np.abs(images[yy, xx])
            if(len(c) > 0):
                img_abs = cv2.cvtColor(img_abs, cv2.COLOR_BGR2RGB)
                plt.imshow(img_abs)
            else:
                plt.imshow(img_abs, cmap = 'gray')

In [ ]:
def imsave(imgdata, filename, title=''):
    imgdata_abs = np.abs(imgdata)
    (h, w, *c) = imgdata.shape
    if(len(c) > 0):
        imgdata_abs = cv2.cvtColor(imgdata_abs, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.axis('off')
    plt.title(title)
    plt.imshow(imgdata_abs, cmap = 'gray')
    plt.savefig('dist/' + filename+'.png')

# PHT

In [ ]:
def translateIndex(index, size):
    return (2 * index - size + 1) / size

In [ ]:
def transformPHT(image, size, maxOrder, progressMessage = ''):

    t_len = 2 * maxOrder + 1
    
    imvec = np.reshape(np.array(image), (size * size, 1))
    transformed = np.array([])
    
    global T_STEP
    try:
        T_STEP
    except NameError:
        T_STEP =  int(maxOrder / 3) + 1

    # GENERATE MATRIX TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((t_len * t_len, 1))
    vecM = vecM.reshape((t_len * t_len, 1))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((1, size * size))
    vecK = vecK.reshape((1, size * size))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX

    updateProgress(0, progressMessage)
    for row in range(int(np.ceil(t_len / T_STEP))):
        start = row * T_STEP * t_len
        end = np.min([row * T_STEP + T_STEP, t_len]) * t_len
        matR = np.repeat(vecR, end - start, axis=0)
        matT = np.repeat(vecT, end - start, axis=0)
        matN = np.repeat(vecN[start:end, :], size * size, axis=1)
        matM = np.repeat(vecM[start:end, :], size * size, axis=1)
        matC = (matR <= 1)
        matF = (np.abs(matN) + np.abs(matM) <= maxOrder)
#         matF = matF * 0 + 1
        matM2 = (np.abs(matM) % 4) != 0

        matW = matM2 * matF * matC * 4 / (np.pi * size * size) * np.exp(-2 * np.pi * matN * matR * matR * 1j - matM * matT * 1j)
        trf_row = np.matmul(matW, imvec)
        transformed = np.append(transformed, trf_row)

        # FREE UP SOME MEMORY - 2
        del matW
        del matN
        del matM
        del matR
        del matT
        del matC
        del matF
        del matM2

        updateProgress((row * T_STEP + T_STEP) / t_len, progressMessage)

    updateProgress(1, progressMessage)
    
    transformed2 = np.reshape(transformed, (t_len, t_len))
    del transformed
    return transformed2

In [ ]:
def inversePHT(momen, size, maxOrder, progressMessage=''):
    
    t_len = 2 * maxOrder + 1

    momenvec = np.reshape(np.array(momen), (t_len * t_len, 1))
    inversed = np.array([])
    
    global I_STEP
    try:
        I_STEP
    except NameError:
        I_STEP =  int(size / 6) + 1


    # GENERATE MATRIX INVERS TRANSFORM

    vecN = np.ones((t_len, t_len), dtype='float32')
    vecM = np.ones((t_len, t_len), dtype='float32')
    for i in range(-maxOrder, maxOrder+1):
        vecN[i + maxOrder, :] = i
        vecM[:, i + maxOrder] = i
    vecN = vecN.reshape((1, t_len * t_len))
    vecM = vecM.reshape((1, t_len * t_len))

    vecI = np.ones((size, size), dtype='float32')
    vecK = np.ones((size, size), dtype='float32')
    for i in range(size):
        vecI[i,:] = i
        vecK[:,i] = i
    vecI = vecI.reshape((size * size, 1))
    vecK = vecK.reshape((size * size, 1))

    vecY = translateIndex(vecI, size)
    vecX = translateIndex(vecK, size)
    vecR = np.sqrt(vecX * vecX + vecY * vecY)
    vecT = np.arctan2(vecY, vecX)

    # FREE UP SOME MEMORY - 1
    del vecI
    del vecK
    del vecY
    del vecX

    updateProgress(0, progressMessage)
    for row in range(int(np.ceil(size / I_STEP))):
        start = row * I_STEP * size
        end = np.min([row * I_STEP + I_STEP, size]) * size
        matR = np.repeat(vecR[start:end, :], t_len * t_len, axis=1)
        matT = np.repeat(vecT[start:end, :], t_len * t_len, axis=1)
        matN = np.repeat(vecN, end - start, axis=0)
        matM = np.repeat(vecM, end - start, axis=0)
        matC = (matR <= 1)
        matF = (np.abs(matN) + np.abs(matM) <= maxOrder)
#         matF = matF * 0 + 1
        matM2 = (np.abs(matM) % 4) != 0

        matV = matM2 * matF * matC * np.exp(2 * np.pi * matN * matR * matR * 1j + matM * matT * 1j)
        inv_row = np.matmul(matV, momenvec)
        inversed = np.append(inversed, inv_row)

        # FREE UP SOME MEMORY - 2
        del matV
        del matN
        del matM
        del matR
        del matT
        del matC
        del matF
        del matM2

        updateProgress((row * I_STEP + I_STEP) / size, progressMessage)

    updateProgress(1, progressMessage)
    
    inversed2 = np.reshape(inversed, (size, size))
    del inversed
    return inversed2

# Watermarking

In [ ]:
def getYFromBGR(imgBGR):
    imgYCrCb = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2YCrCb)
    return imgYCrCb[:,:,0]

def putYToBGR(imgY, imgBGR):
    imgYCrCb = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2YCrCb)
    imgY = np.abs(imgY).astype(np.uint8)
    imgYCrCb[:,:,0] = imgY
    imgBGR2 = cv2.cvtColor(imgYCrCb, cv2.COLOR_YCrCb2BGR)
    return imgBGR2

In [ ]:
def calcBE(W1, W2):
    return np.sum(np.abs(np.abs(W1) - np.abs(W2)))

In [ ]:
N_SCRAMBLE = 13

def scramble(logo, n=N_SCRAMBLE):
    if(n==0): return logo
    (h, w) = logo.shape
    newLogo = np.zeros(logo.shape)
    for y in range(h):
        for x in range(w):
            val = logo[y, x]
            newLogo[(y + x) % h, (y + 2 * x) % w] = val
    return scramble(newLogo, n-1)

def unscramble(logo, n=N_SCRAMBLE):
    if(n==0): return logo
    (h, w) = logo.shape
    newLogo = np.zeros(logo.shape)
    for y in range(h):
        for x in range(w):
            val = logo[y, x]
            newLogo[(2 * y - x) % h, (- y + x) % w] = val
    return unscramble(newLogo, n-1)

In [ ]:
C_SCRAMBLE = 15

def scramble1D(array, n=C_SCRAMBLE):
    if(n==0): return array
    (l, ) = array.shape
    f1 = np.array([], dtype=bool)
    f2 = np.array([], dtype=bool)
    for i in range(l):
        if(i % 2 == 0):
            f1 = np.append(f1, True)
            f2 = np.append(f2, False)
        else:
            f1 = np.append(f1, False)
            f2 = np.append(f2, True)
    arr1 = array[f1]
    arr2 = array[f2]
    result = np.append(arr1, arr2)
    result = np.roll(result, 1)
    del f1, f2, arr1, arr2
    return scramble1D(result, n-1)

def unscramble1D(array, n=C_SCRAMBLE):
    if(n==0): return array
    array = np.roll(array, -1)
    (l, ) = array.shape
    l2 = np.ceil(l / 2)
    arr1 = array[0:int(l2)]
    arr2 = array[int(l2):]
    result = np.array([])
    for i in range(l):
        if(i % 2 == 0):
            result = np.append(result, arr1[int(i/2)])
        else:
            result = np.append(result, arr2[int(i/2)])
    del arr1, arr2
    return unscramble1D(result, n-1)

In [ ]:
SN = 0 # Start of n
MIN_D = 0 # Min Diagonal
MAX_D = 1 # Max Diagonal
MIN_L = 0
SCR1D = True

def watermarkEmbed(logo, p):
    logo_sc = scramble(logo)
    p2 = 2 * p + 1
    count = 0
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= MIN_D * p and np.abs(l) + n <= MAX_D * p and l > MIN_L):
                count += 1
                
    logo_f = np.copy(logo_sc).flatten() # logo flattened
    (ll,) = logo_f.shape
    
    logo_s = np.zeros((count,)) - 1 # logo scrambled
    logo_s[0:ll] = logo_f
    if SCR1D:
        logo_s = scramble1D(logo_s)
    
    container = np.zeros((p2,p2))
    containerFilter = np.zeros((p2, p2))
    index = 0
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= MIN_D * p and np.abs(l) + n <= MAX_D * p and l > MIN_L):
                container[p + n, m] = logo_s[index]
                containerFilter[p + n, m] = 1
                index += 1
    
    containerFilter[np.where(container == -1)] = 0
    container[np.where(container == -1)] = 0
    
    container = container + np.rot90(np.rot90(container))
    containerFilter = containerFilter + np.rot90(np.rot90(containerFilter))
    
    return (container, containerFilter, index)

def watermarkExtract(momen_bw, p, ll):
    p2 = 2 * p + 1
    logo_f = np.array([])
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= MIN_D * p and np.abs(l) + n <= MAX_D * p and l > MIN_L):
                val = momen_bw[p + n, m]
                logo_f = np.append(logo_f, val)
    
    if SCR1D:
        logo_f = unscramble1D(logo_f)
    logo_f = logo_f[0:ll*ll]
    logo_f = logo_f.reshape((ll,ll))
    return unscramble(logo_f)

# Attacks

In [ ]:
# Attacks

# Rotation
def attackRotation(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# Resize
def attackResize(image, size):
    result = cv2.resize(image, (size, size))
    return result

# Translation
def attackTranslate(image, px, py):
    (h, w, c) = image.shape
    dx = int(px * w)
    dy = int(py * h)
    result = np.zeros((h, w, c)).astype(np.uint8)
    if(dx > 0):
        result[:,dx:w,:] = image[:,0:w-dx,:]
    else:
        result[:,0:w+dx,:] = image[:,-dx:w,:]
    result2 = np.zeros((h, w, c)).astype(np.uint8)
    if(dy > 0):
        result2[dy:h,:,:] = result[0:h-dy,:,:]
    else:
        result2[0:h+dy,:,:] = result[-dy:h,:,:]
    return result2

# Translation
def attackTranslateByPixel(image, dx, dy):
    (h, w, c) = image.shape
    result = np.zeros((h, w, c)).astype(np.uint8)
    if(dx > 0):
        result[:,dx:w,:] = image[:,0:w-dx,:]
    else:
        result[:,0:w+dx,:] = image[:,-dx:w,:]
    result2 = np.zeros((h, w, c)).astype(np.uint8)
    if(dy > 0):
        result2[dy:h,:,:] = result[0:h-dy,:,:]
    else:
        result2[0:h+dy,:,:] = result[-dy:h,:,:]
    return result2

# Crop
def attackCrop(image, tp, rp, bp, lp):
    (h, w, c) = image.shape
    dt = int(h*tp)
    db = int(h*bp)
    dl = int(w*lp)
    dr = int(w*rp)
    result = np.copy(image)
    result[0:dt,:,:] = np.zeros((dt, w, c)).astype(np.uint8)
    result[h-db:h,:,:] = np.zeros((db, w, c)).astype(np.uint8)
    result[:,0:dl,:] = np.zeros((h, dl, c)).astype(np.uint8)
    result[:,w-dl:w,:] = np.zeros((h, dr, c)).astype(np.uint8)
    return result

# Blur
def attackBlur(image, kernelSize):
    result = cv2.blur(image, (kernelSize, kernelSize))
    return result

# Noise
def attackRandomNoise(image, n_p):
    result = np.copy(image)
    for i, row in enumerate(image):
        for j, val in enumerate(row):
            if(np.random.randint(low=0, high=100) < n_p * 100 / 2):
                result[i,j] = 0
    for i, row in enumerate(image):
        for j, val in enumerate(row):
            if(np.random.randint(low=0, high=100) < n_p * 100 / 2):
                result[i,j] = 255
    return result

# JPEG Compression
def attackJPG(image, Q=95):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), Q]
    result, encimg = cv2.imencode('.jpg', image, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg

# Inverse Translation

In [ ]:
def kp2pt(kp):
    (px, py) = kp.pt
    return (int(px), int(py))

def getPairs(keypointmathces, keypoints1, keypoints2):
    pairs = []
    for mt in keypointmathces:
        p1 = keypoints1[mt.queryIdx]
        p2 = keypoints2[mt.trainIdx]
        pairs.append((kp2pt(p1), kp2pt(p2)))
    return pairs

def calcDist(pa, pb):
    (pax, pay) = pa
    (pbx, pby) = pb
    dx = pax - pbx
    dy = pay - pby
    return np.sqrt(dx * dx + dy * dy)

def addTupple(t1, t2):
    (t1x, t1y) = t1
    (t2x, t2y) = t2
    return (t1x + t2x, t1y + t2y)

def calcError(keypointmathces, keypoints1, keypoints2, center1, center2, s):
    pairs = getPairs(keypointmathces, keypoints1, keypoints2)
    sum = 0
    for (p1, p2) in pairs:
        d1 = calcDist(p1, center1)
        d2 = s * calcDist(p2, center2)
        sum += abs(d1 - d2)
    return sum / len(pairs)

In [ ]:
def optimizeCenter(keypointmathces, keypoints1, keypoints2, c1, c2):
    dx = 0
    dy = 0
    s = 1
    d_step = 0.1
    s_step = 0.0005
    N_STEP = 10000
    
    err = 0
    
    for i in range(N_STEP):
        
        if(i%3 == 0):
            e1 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx + d_step, c2 + dy), s)
            e2 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx - d_step, c2 + dy), s)
            if(e1 < e2):
                err = e1
                dx += d_step
            else:
                err = e2
                dx -= d_step
        elif(i%3 == 1):
            e1 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx, c2 + dy + d_step), s)
            e2 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx, c2 + dy - d_step), s)
            if(e1 < e2):
                err = e1
                dy += d_step
            else:
                err = e2
                dy -= d_step
        else:
            e1 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx, c2 + dy), s + s_step)
            e2 = calcError(keypointmathces[:], keypoints1, keypoints2, (c1, c1), (c2 + dx, c2 + dy), s - s_step)
            if(e1 < e2):
                err = e1
                s += s_step
            else:
                err = e2
                s -= s_step
                
    return (int(np.round(-dx)), int(np.round(-dy)), s)

In [ ]:
def correctTranslate(original, attacked):
    
    # Initiate ORB detector
    orb = cv2.ORB_create()
    
    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(original,None)
    kp2, des2 = orb.detectAndCompute(attacked,None)
    
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors.
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    (n1, *_) = original.shape
    (n2, *_) = attacked.shape
    
    return optimizeCenter(matches[:50], kp1, kp2, (n1 - 1) / 2, (n2 - 1) / 2)